In [2]:
import pandas as pd
import numpy as np
import ast 
import re
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download("punkt")
nltk.download("stopwords")
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Abhi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Abhi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
credit_data=pd.read_csv("D:\Recomendor system PW\data\\tmdb_5000_credits.csv")

In [20]:
movie_data=pd.read_csv("D:\Recomendor system PW\data\\tmdb_5000_movies.csv")

In [22]:
movies=movie_data.merge(credit_data,on="title")

In [23]:
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


```
Feature Engineering:-

In [25]:
movies.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'movie_id', 'cast', 'crew'],
      dtype='object')

In [26]:
movies=movies[["genres","id","keywords","original_language","title","overview","popularity","cast", "crew"]]

In [28]:
movies.head()

,genres,id,keywords,original_language,title,overview,popularity,cast,crew
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [29]:
movies.duplicated().sum()

0

In [30]:
movies.isnull().sum()

genres               0
id                   0
keywords             0
original_language    0
title                0
overview             3
popularity           0
cast                 0
crew                 0
dtype: int64

In [32]:
movies=movies.dropna()

In [33]:
movies.head()

,genres,id,keywords,original_language,title,overview,popularity,cast,crew
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [36]:
movies.iloc[0]["genres"]
movies.iloc[0]["keywords"]

'[{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}, {"id": 3386, "name": "space war"}, {"id": 3388, "name": "space colony"}, {"id": 3679, "name": "society"}, {"id": 3801, "name": "space travel"}, {"id": 9685, "name": "futuristic"}, {"id": 9840, "name": "romance"}, {"id": 9882, "name": "space"}, {"id": 9951, "name": "alien"}, {"id": 10148, "name": "tribe"}, {"id": 10158, "name": "alien planet"}, {"id": 10987, "name": "cgi"}, {"id": 11399, "name": "marine"}, {"id": 13065, "name": "soldier"}, {"id": 14643, "name": "battle"}, {"id": 14720, "name": "love affair"}, {"id": 165431, "name": "anti war"}, {"id": 193554, "name": "power relations"}, {"id": 206690, "name": "mind and soul"}, {"id": 209714, "name": "3d"}]'

In [54]:
def get_values(value):
    values=ast.literal_eval(value)
    lst=[]
    for dictionary in values:
        lst.append(dictionary["name"])
    return lst

In [56]:
movies["genres"]=movies["genres"].apply(lambda x: get_values(x))

In [57]:
movies["keywords"]=movies["keywords"].apply(lambda x: get_values(x))

In [60]:
def get_cast(value):
    values=ast.literal_eval(value)
    lst=[]
    countr=0
    
    
    
    for dictionary in values:
        if countr < 5:
            lst.append(dictionary["name"])
            countr +=1
            
        else:
            break
            
            
    return lst

In [61]:
movies["cast"]=movies["cast"].apply(lambda x: get_cast(x))

In [62]:
movies.head(1)

,genres,id,keywords,original_language,title,overview,popularity,cast,crew
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[culture clash, future, space war, space colon...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[Sam Worthington, Zoe Saldana, Sigourney Weave...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [64]:
movies.iloc[0]["cast"]

['Sam Worthington',
 'Zoe Saldana',
 'Sigourney Weaver',
 'Stephen Lang',
 'Michelle Rodriguez']

In [ ]:
def get_values(value):
    values=ast.literal_eval(value)
    lst=[]
    for dictionary in values:
        if dictionary["job"]=="Director"
        lst.append(dictionary["name"])
        break
    return lst

In [65]:
movies["crew"]=movies["crew"].apply(lambda x: get_values(x))

In [66]:
movies.head(1)

,genres,id,keywords,original_language,title,overview,popularity,cast,crew
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[culture clash, future, space war, space colon...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[Sam Worthington, Zoe Saldana, Sigourney Weave...","[Stephen E. Rivkin, Rick Carter, Christopher B..."


In [68]:
movies["overview"]=movies["overview"].apply(lambda x: x.split(" "))

In [70]:
movies.head(1)

,genres,id,keywords,original_language,title,overview,popularity,cast,crew
0,"[Action, Adventure, Fantasy, Science Fiction]",19995,"[culture clash, future, space war, space colon...",en,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...",150.437577,"[Sam Worthington, Zoe Saldana, Sigourney Weave...","[Stephen E. Rivkin, Rick Carter, Christopher B..."


In [72]:
movies["tag"]=movies["genres"]+movies["keywords"]+movies["cast"]+movies["crew"]+movies["overview"]

In [75]:
movies_tag=movies[["title","id","original_language","popularity","tag"]]

In [77]:
movies_tag.head()

,title,id,original_language,popularity,tag
0,Avatar,19995,en,150.437577,"[Action, Adventure, Fantasy, Science Fiction, ..."
1,Pirates of the Caribbean: At World's End,285,en,139.082615,"[Adventure, Fantasy, Action, ocean, drug abuse..."
2,Spectre,206647,en,107.376788,"[Action, Adventure, Crime, spy, based on novel..."
3,The Dark Knight Rises,49026,en,112.312950,"[Action, Crime, Drama, Thriller, dc comics, cr..."
4,John Carter,49529,en,43.926995,"[Action, Adventure, Science Fiction, based on ..."


In [79]:
movies_tag["tag"][0]

['Action',
 'Adventure',
 'Fantasy',
 'Science Fiction',
 'culture clash',
 'future',
 'space war',
 'space colony',
 'society',
 'space travel',
 'futuristic',
 'romance',
 'space',
 'alien',
 'tribe',
 'alien planet',
 'cgi',
 'marine',
 'soldier',
 'battle',
 'love affair',
 'anti war',
 'power relations',
 'mind and soul',
 '3d',
 'Sam Worthington',
 'Zoe Saldana',
 'Sigourney Weaver',
 'Stephen Lang',
 'Michelle Rodriguez',
 'Stephen E. Rivkin',
 'Rick Carter',
 'Christopher Boyes',
 'Christopher Boyes',
 'Mali Finn',
 'James Horner',
 'James Cameron',
 'James Cameron',
 'James Cameron',
 'James Cameron',
 'James Cameron',
 'Andrew Menzies',
 'Jill Brooks',
 'Margery Simkin',
 'Kevin Ishioka',
 'Dick Bernstein',
 'Shannon Mills',
 'Dennie Thorpe',
 'Jana Vance',
 'Deborah Lynn Scott',
 'Jon Landau',
 'Sean Haworth',
 'Kim Sinclair',
 'Kim Sinclair',
 'Richard F. Mays',
 'Laeta Kalogridis',
 'Mayes C. Rubeo',
 'Mauro Fiore',
 'Scott Herbertson',
 'Woody Schultz',
 'Linda DeVetta',


In [91]:
def process_tag(value):
    ps=PorterStemmer()
    
    
    tags=[ps.stem(word) for word in value if word not in stopwords.words("english")]
    
    text=" ".join(tags)

    text=re.sub(r"[^\w\s]","",text.lower()) 
    return text

    
    

In [92]:
movies_tag["prec_tag"]=movies_tag["tag"].apply(lambda x: process_tag(x))

In [93]:
movies_tag["prec_tag"][0]

'action adventur fantasi science fict culture clash futur space war space coloni societi space travel futurist romanc space alien tribe alien planet cgi marin soldier battl love affair anti war power rel mind and soul 3d sam worthington zoe saldana sigourney weav stephen lang michelle rodriguez stephen e rivkin rick cart christopher boy christopher boy mali finn james horn james cameron james cameron james cameron james cameron james cameron andrew menzi jill brook margery simkin kevin ishioka dick bernstein shannon mil dennie thorp jana v deborah lynn scott jon landau sean haworth kim sinclair kim sinclair richard f may laeta kalogridi mayes c rubeo mauro fior scott herbertson woody schultz linda devetta linda devetta richard bluck simon bright richard martin steve r moor john refoua karl j martin chiling lin ilram choi steven qual carla mey nick bassett jill cormack andy mclaren terry notari garrett warren jonathan rothbart stefan dech todd cherniawski miranda riv robert stromberg jo

In [97]:
cv=CountVectorizer(max_features=5000,stop_words="english")

In [98]:
vector=cv.fit_transform(movies_tag["prec_tag"]).toarray()

In [100]:
vector.shape

(4806, 5000)

In [101]:
cos_sim=cosine_similarity(vector)

In [102]:
cos_sim.shape

(4806, 4806)

In [103]:
cos_sim[0]

array([1.        , 0.18172216, 0.26594732, ..., 0.07870418, 0.03396413,
       0.05856369])

In [104]:
def recomendor(movie):
    movie_index=movies_tag[movies_tag["title"]==movie].index[0]
    
    
    dist=cos_sim[movie_index]
    
    movies_list=sorted(list(enumerate(dist)),reverse=True,key=lambda x:x[1])[1:6]
    
    for movie in movies_list:
        print(movies_tag.iloc[[movie[0]]].title)
    

In [108]:
recomendor("Superman Returns")

203    X2
Name: title, dtype: object
64    X-Men: Apocalypse
Name: title, dtype: object
511    X-Men
Name: title, dtype: object
46    X-Men: Days of Future Past
Name: title, dtype: object
48    Jack the Giant Slayer
Name: title, dtype: object


In [112]:
movies_tag["title"][10]

'Superman Returns'

In [113]:
import pickle
pickle.dump(cos_sim,open("D:\Recomendor system PW\models\similarity.pickle","wb"))
pickle.dump(movies_tag.to_dict(),open("D:\Recomendor system PW\models\movies_tag.pickle","wb"))